# Introduction

In this notebook, we will implement the standard method of Collaborative Filtering with the Pareto Dominance. It is inspired by this paper: 
http://www.sciencedirect.com/science/article/pii/S0020025513002004

In [1]:
# Import the fucking useful libraries =)
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import itertools
from joblib import Parallel, delayed
import pickle

%matplotlib inline
%load_ext autoreload
%autoreload 2

# There's a lot of columns in the DF. 
# Therefore, we add this option so that we can see more columns
pd.options.display.max_columns = 100

In [2]:
# Define here the data set
dataset = '../data/data_train.csv'

# Load the data

In [3]:
data = pd.read_csv(dataset)
data['UserID'] = data['Id'].apply(lambda x: int(x.split('_')[0][1:])-1)
data['MovieID'] = data['Id'].apply(lambda x: int(x.split('_')[1][1:])-1)
data.head()


,Id,Prediction,UserID,MovieID
0,r44_c1,4,43,0
1,r61_c1,3,60,0
2,r67_c1,4,66,0
3,r72_c1,3,71,0
4,r86_c1,5,85,0


In [4]:
data.shape

(1176952, 4)

# Define some Variables

In here, we will define some sets:
* $U = \left\{u\in \mathbb{N} \mid 0 \leq u \leq l-1 \right\}$, the set of users
* $I = \left\{i \in \mathbb{N} \mid 0 \leq i \leq m-1 \right\}$, the set of items
* $V = \left\{v \in \mathbb{N} \mid min \leq v \leq max \right\} \cup \left\{ \texttt{NaN} \right\}$, the set of possible ratings
* $R_u = \left\{(i,v) \mid i \in I, v \in V \right\}$, ratings of user u
* $I_u = \left\{i \in I \mid r_{u,i} \neq \cdot \right\}$, set of items rated by user $u$

In [5]:
U = np.sort(data['UserID'].unique())
l = len(U)
l


10000

In [6]:
I = np.sort(data['MovieID'].unique())
m = len(I)
m

1000

In [7]:
min_ = 1
max_ = 5

In [8]:
# To create R_u, we create a matrix with User as first entry and item as second entry
R_u = -1*np.ones((l, m))

for lines in range(len(data)):
    R_u[data['UserID'][lines], data['MovieID'][lines]] = data['Prediction'][lines]
    
# Replace with NAN
R_u[R_u==-1] = np.inf

In [9]:
R_u.shape

(10000, 1000)

In [10]:
# To create I_u, we create a matrix with User as first entry and item as second entry
I_u = []
for line in range(R_u.shape[0]):
    I_u.append(I[~np.isinf(R_u[line])])
    
#I_u

In [11]:
def abs_diff(x, y, i, R):
    """
        Compute Absolute difference between the ratings given by user x and user y to the item i
    
        IN:
            x: A user
            y: A second user
            i: An item
            R: Matrix of ratings
    
    """
        
    return np.abs(R[x, i]-R[y, i])

In [12]:
def get_all_distances(U, u, I_u, R):
    
    D = np.zeros((len(U), len(I_u[u])))
    
    for usr in U:
        if usr != u:
            D[usr,:] = abs_diff(u, usr, I_u[u], R)
            
    return D

In [13]:
def get_matrix_dominance(U, D):
    M = np.zeros((len(U), len(U)))
    for x in range(len(U)):
        M[x, :] = ((D[x]<=D).all(axis=1)*(D[x]<D).any(axis=1))
        
    return M

In [14]:
def sets_dominated_dominator(U, u, I_u, R, M):
    """
        Returns the set of dominated users and the set of non-dominated user w.r.t user u
        
        IN:
            U: Set of users
            u: A specific user
            I_u: List of items (Only the one that were rated)
            R: Matrix of Ratings
            
        OUT:
            C_u: Set of non-dominated users w.r.t. user u (candidates)
            D_u: Set of dominated users
    """
    
    D_u = []
    C_u = []
            
    for idx, usr in enumerate(np.delete(U, u)):
        if (M[:u:,usr] == 0).all():
            C_u.append(usr)
        else:
            D_u.append(usr)

    return C_u, D_u

In [15]:
def create_Cu(U, u, I_u, R_u):
    D = get_all_distances(U, u, I_u, R_u)
    M = get_matrix_dominance(U, D)
    C_u, D_u = sets_dominated_dominator(U, u, I_u, R_u, M)
    filename = "./pickles/C_" + str(u) + ".pickle"
    pickle.dump(C_u, open(filename, "wb"))
    print("C_%i created"%(u))

In [ ]:
%%time
Parallel(n_jobs=8)(delayed(create_Cu)(U, u, I_u, R_u) for u in U[7700:])

C_6930 created


In [8]:
len(pickle.load(open("./pickles/C_.pickle", "rb")))

9322